# <center> Лабораторна робота №11. Налаштування гіперпараметрів регресійних моделей для оцінювання якості вина


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

У завданні буде використано набір даних про якість білого вина(репозиторій UCI)
archive.ics.uci.edu/ml/machine-learning-databases/wine-quality.
Завантажте дані

In [4]:
# !mkdir -p data
# !wget -P data -c https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

data = pd.read_csv("data/winequality-white.csv", sep=";")
display(data.sample(10))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2093,6.6,0.22,0.53,15.10,0.052,22.0,136.0,0.99860,2.94,0.35,9.4,5
1899,6.7,0.44,0.31,1.90,0.030,41.0,104.0,0.99000,3.29,0.62,12.6,7
1030,7.1,0.20,0.41,2.10,0.054,24.0,166.0,0.99480,3.48,0.62,10.5,6
3019,7.2,0.32,0.30,8.25,0.020,14.0,104.0,0.99362,2.99,0.44,11.4,6
867,7.8,0.18,0.46,12.60,0.042,41.0,143.0,1.00000,3.24,0.76,8.5,8
474,5.5,0.16,0.22,4.50,0.030,30.0,102.0,0.99380,3.24,0.36,9.4,6
1419,7.6,0.14,0.74,1.60,0.040,27.0,103.0,0.99160,3.07,0.40,10.8,7
2578,5.8,0.30,0.27,1.70,0.014,45.0,104.0,0.98914,3.40,0.56,12.6,7
1776,6.6,0.25,0.35,14.00,0.069,42.0,163.0,0.99900,3.56,0.47,9.8,5
564,6.8,0.51,0.30,4.20,0.066,38.0,165.0,0.99450,3.20,0.42,9.1,5


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Відокремте цільову змінну, розділіть навчальну вибірку у відношенні 7:3 (30% - під задишену вибірку, нехай random_state=17) і нормалізуйте дані за допомогою StandartScaler


In [8]:
y = data["quality"]
data.drop("quality", axis=1, inplace=True)
X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)


## Лінійна регресія


Навчіть просту лінійну модель регресії

In [9]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

> **Питання 1 : Які середньоквадратичні помилки лінійної регресії на навчальній і відкоаденій вибірках ?**

In [12]:
y_train_pred = linreg.predict(X_train_scaled)
y_holdout_pred = linreg.predict(X_holdout_scaled)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_holdout, y_holdout_pred)

print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.584


Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина (врахуйте, що великі за модулем негативні значення коефіцієнтів теж говорять про сильний вплив). Створіть для цього новий невеликий DataFrame.
> **Питання 2 : Яку ознаку лінійна регресія вважає найбільш впливовою на якість вина?**




In [13]:
linreg_coef = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': linreg.coef_})
linreg_coef['Absolute Coefficient'] = linreg_coef['Coefficient'].abs()
linreg_coef.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)
display(linreg_coef)

,Feature,Coefficient,Absolute Coefficient
7,density,-0.665720,0.665720
3,residual sugar,0.538164,0.538164
1,volatile acidity,-0.192260,0.192260
8,pH,0.150036,0.150036
10,alcohol,0.129533,0.129533
0,fixed acidity,0.097822,0.097822
9,sulphates,0.062053,0.062053
5,free sulfur dioxide,0.042180,0.042180
6,total sulfur dioxide,0.014304,0.014304
4,chlorides,0.008127,0.008127


## Lasso-регресія

**Навчіть Lasso-регресію з невеликим коефіцієнтом alpha=0,01 (слабка регуляризація). Нехай знову random_state=17.**


In [14]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Подивіться на коефіцієнти моделі і ранжуйте ознаки за впливом на якість вина. Яка ознака "відпала" першою, тобто найменш важлива для пояснення цільової змінної в моделі Lasso?**

In [15]:
lasso1_coef = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': lasso1.coef_})
lasso1_coef['Absolute Coefficient'] = lasso1_coef['Coefficient'].abs()
lasso1_coef.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)
display(lasso1_coef)

,Feature,Coefficient,Absolute Coefficient
10,alcohol,0.322425,0.322425
3,residual sugar,0.256363,0.256363
7,density,-0.235492,0.235492
1,volatile acidity,-0.188479,0.188479
8,pH,0.067277,0.067277
5,free sulfur dioxide,0.043088,0.043088
9,sulphates,0.029722,0.029722
4,chlorides,-0.002747,0.002747
0,fixed acidity,-0.000000,0.000000
2,citric acid,-0.000000,0.000000


**Тепер визначте краще значення alpha в процесі 5-кратної крос-валідації. Використовуйте LassoCV і random_state = 17.**

In [16]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [17]:
lasso_cv.alpha_

np.float64(0.0002833096101839324)

Виведіть коефіцієнти "кращого" Lasso в порядку зменшення впливу на якість вина.
> **Питання 3: Яка ознака "занулилася першою" в налаштованій моделі LASSO?**

In [18]:
lasso_cv_coef = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': lasso_cv.coef_})
lasso_cv_coef['Absolute Coefficient'] = lasso_cv_coef['Coefficient'].abs()
lasso_cv_coef.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)
display(lasso_cv_coef)

,Feature,Coefficient,Absolute Coefficient
7,density,-0.648161,0.648161
3,residual sugar,0.526883,0.526883
1,volatile acidity,-0.192049,0.192049
8,pH,0.146549,0.146549
10,alcohol,0.137115,0.137115
0,fixed acidity,0.093295,0.093295
9,sulphates,0.060939,0.060939
5,free sulfur dioxide,0.042698,0.042698
6,total sulfur dioxide,0.012969,0.012969
4,chlorides,0.006933,0.006933


**Оцініть середньоквадратичну помилку моделі на навчальній і тестовій вибірках.**

> **Питання 4 : Які середньоквадратичні помилки налаштованої LASSO-регресії на навчальній і відкладеній вибірках?**

In [19]:
y_train_pred_lasso = lasso_cv.predict(X_train_scaled)
y_holdout_pred_lasso = lasso_cv.predict(X_holdout_scaled)

mse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso)
mse_test_lasso = mean_squared_error(y_holdout, y_holdout_pred_lasso)

print("Mean squared error (train): %.3f" % mse_train_lasso)
print("Mean squared error (test): %.3f" % mse_test_lasso)

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Випадковий ліс

**Навчіть випадковий ліс з параметрами "з коробки", фіксуючи тільки random_state=17.**

In [20]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

> **Питання 5 : Які середньоквадратичні помилки випадкового лысу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error' і іншими параметрами за замовчуванням) і відкладеній вибірках?**

In [21]:
# Прогнози на навчальній вибірці
y_train_pred_forest = forest.predict(X_train_scaled)
mse_train_forest = mean_squared_error(y_train, y_train_pred_forest)

# Оцінка на крос-валідації
cv_scores_forest = cross_val_score(forest, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
mse_cv_forest = -cv_scores_forest.mean()

# Прогнози на відкладеній вибірці
y_holdout_pred_forest = forest.predict(X_holdout_scaled)
mse_test_forest = mean_squared_error(y_holdout, y_holdout_pred_forest)

print("Mean squared error (train): %.3f" % mse_train_forest)
print("Mean squared error (cv): %.3f" % mse_cv_forest)
print("Mean squared error (test): %.3f" % mse_test_forest)

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Налаштуйте параметри min_samples_leaf і max_depth за допомогою GridSearchCV і знову перевірте якість моделі на крос-валідації і на відкладеній вибірках.**

In [22]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6, 12))}

locally_best_forest = GridSearchCV(forest, forest_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)

print("Best parameters:", locally_best_forest.best_params_)
print("Best cross-validation score (negative MSE):", locally_best_forest.best_score_)


Best parameters: {'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1}
Best cross-validation score (negative MSE): -0.39773288191505934


In [23]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 np.float64(-0.39773288191505934))

**Нажал результати GridSearchCV в повному не відтворювані (можуть відрізнятися на різних платформах навіть при фіксованому random_state). Тому навчіть ліс з параметрами max_depth=19, max_features=7, i min_samples_leaf=1 (краще в моэму випадку).**
> **Питання 6 : Які середньоквадратичні помилки налаштованого випадкового лісу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error') і на відкладеній вибірках?**


In [24]:
# Навчіть випадковий ліс з параметрами max_depth=19, max_features=7, min_samples_leaf=1
tuned_forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
tuned_forest.fit(X_train_scaled, y_train)

# Прогнози на навчальній вибірці
y_train_pred_tuned_forest = tuned_forest.predict(X_train_scaled)
mse_train_tuned_forest = mean_squared_error(y_train, y_train_pred_tuned_forest)

# Оцінка на крос-валідації
cv_scores_tuned_forest = cross_val_score(tuned_forest, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
mse_cv_tuned_forest = -cv_scores_tuned_forest.mean()

# Прогнози на відкладеній вибірці
y_holdout_pred_tuned_forest = tuned_forest.predict(X_holdout_scaled)
mse_test_tuned_forest = mean_squared_error(y_holdout, y_holdout_pred_tuned_forest)

print("Mean squared error (train): %.3f" % mse_train_tuned_forest)
print("Mean squared error (cv): %.3f" % mse_cv_tuned_forest)
print("Mean squared error (test): %.3f" % mse_test_tuned_forest)

Mean squared error (train): 0.058
Mean squared error (cv): 0.404
Mean squared error (test): 0.370


**Оцініть важливість ознак за допомогою випадкового лісу.**
>**Питання 7 : Яка ознака виявилася найінформативнішою в налаштованій моделі випадкового лісу?**

In [25]:
rf_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': tuned_forest.feature_importances_})
rf_importance.sort_values(by='Importance', ascending=False, inplace=True)
display(rf_importance)

,Feature,Importance
10,alcohol,0.214825
1,volatile acidity,0.117627
5,free sulfur dioxide,0.113509
7,density,0.086530
8,pH,0.072646
3,residual sugar,0.072529
6,total sulfur dioxide,0.071644
4,chlorides,0.067819
0,fixed acidity,0.061873
2,citric acid,0.061500


**Зробіть висновки про якість моделей і оцінки впливу ознак на якість вина за допомогою цих трьох моделей**

```markdown
**Висновки про якість моделей і оцінки впливу ознак на якість вина**

1. **Лінійна регресія**:
    - Середньоквадратична помилка на навчальній вибірці: 0.558
    - Середньоквадратична помилка на відкладеній вибірці: 0.584
    - Найбільш впливова ознака: `density` (коефіцієнт: -0.666)

2. **Lasso-регресія**:
    - Середньоквадратична помилка на навчальній вибірці: 0.558
    - Середньоквадратична помилка на відкладеній вибірці: 0.583
    - Найбільш впливова ознака: `density` (коефіцієнт: -0.648)
    - Ознаки, які занулилися першими: `fixed acidity`, `citric acid`, `total sulfur dioxide`

3. **Випадковий ліс**:
    - Середньоквадратична помилка на навчальній вибірці: 0.053
    - Середньоквадратична помилка на крос-валідації: 0.414
    - Середньоквадратична помилка на відкладеній вибірці: 0.372
    - Найбільш впливова ознака: `alcohol` (важливість: 0.215)

4. **Налаштований випадковий ліс**:
    - Середньоквадратична помилка на навчальній вибірці: 0.058
    - Середньоквадратична помилка на крос-валідації: 0.404
    - Середньоквадратична помилка на відкладеній вибірці: 0.370
    - Найбільш впливова ознака: `alcohol` (важливість: 0.215)

**Загальні висновки**:
- Лінійна регресія та Lasso-регресія показують схожі результати за середньоквадратичною помилкою, але Lasso-регресія дозволяє визначити менш важливі ознаки, зануляючи їхні коефіцієнти.
- Випадковий ліс показує кращі результати за середньоквадратичною помилкою на відкладеній вибірці порівняно з лінійною та Lasso-регресією.
- Налаштований випадковий ліс має трохи кращі результати, ніж випадковий ліс з параметрами "з коробки".
- Найбільш впливовою ознакою для випадкового лісу є `alcohol`, тоді як для лінійної та Lasso-регресії це `density`.
```